# CYBERML - Project

## Importation du dataset

In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import seaborn as sns
import numpy as np

In [682]:
# file1_path = "../SWaT.A7_June 2020/22June2020 (1).xlsx"
# file2_path = "../SWaT.A7_June 2020/22June2020 (2).xlsx"
# file3_path = "../SWaT.A7_June 2020/29June2020 (1).xlsx"
# file4_path = "../SWaT.A7_June 2020/29June2020 (2).xlsx"

# df1 = pd.read_excel(file1_path)
# df2 = pd.read_excel(file2_path)
# df3 = pd.read_excel(file3_path, usecols=range(61))
# df4 = pd.read_excel(file4_path)
# print(df1.shape)
# print(df2.shape)
# print(df3.shape)
# print(df4.shape)

In [683]:
# print(df1.columns.to_list())
# print(df2.columns.to_list())
# print(df3.columns.to_list())
# print(df4.columns.to_list())

In [684]:
# common_columns = list(set(df1.columns) & set(df2.columns) & set(df3.columns) & set(df4.columns))
# # columns in df1 but not in common_columns
# missing_columns = list(set(df1.columns) - set(common_columns))
# print(missing_columns)
# print(len(missing_columns))
# print(len(common_columns))

# print(f"Missing columns : {list(set(df1.columns) ^ set(df2.columns) ^ set(df3.columns) ^ set(df4.columns))}")

In [685]:
# df1 = df1[common_columns]
# df2 = df2[common_columns]
# df3 = df3[common_columns]
# df4 = df4[common_columns]
# print(df1.columns.to_list())
# print(df2.columns.to_list())
# print(df3.columns.to_list())
# print(df4.columns.to_list())

In [686]:
# if list(df1.columns) != list(df2.columns) or list(df1.columns) != list(df3.columns) or list(df1.columns) != list(df4.columns):
#     print("Les colonnes des deux fichiers ne correspondent pas.")
#     print("Colonnes dans le fichier 1 :", df1.columns)
#     print("Colonnes dans le fichier 2 :", df2.columns)
#     print("Colonnes dans le fichier 3 :", df3.columns)
#     print("Colonnes dans le fichier 4 :", df4.columns)
# else:
#     print("Les colonnes des deux fichiers correspondent.")
#     df_combined = pd.concat([df1, df2, df3, df4], axis=0, ignore_index=True)

#     # Vérifier le nombre de lignes et de colonnes du dataset combiné
#     print(f"Dataset combiné : {df_combined.shape} lignes et colonnes")

#     # Sauvegarder le dataset combiné dans un fichier CSV
#     output_path = "../SWaT.A7_June 2020/df_combined.csv"
#     df_combined.to_csv(output_path, index=False)

#     print(f"Dataset fusionné sauvegardé dans : {output_path}")

In [2]:
df = pd.read_csv("../SWaT.A7_June 2020/df_combined.csv", low_memory=False)

In [ ]:
df.shape

In [ ]:
df.tail()

In [5]:
df = df.drop(df.tail(1).index)

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()

In [ ]:
pd.set_option('display.max_columns', None)
df.columns

In [10]:
cols_numeric = cols_to_convert = df.columns.difference(['t_stamp'])
df[cols_numeric] = df[cols_numeric].apply(pd.to_numeric, errors='coerce')

In [ ]:
df.describe()

In [ ]:
df.dtypes

### Pré-traitement des données

In [ ]:
df = df.dropna()
df.isnull().sum()

In [ ]:
columns_to_encode = [column for column in df.columns if df[column].nunique() < 20]
print(columns_to_encode)
print(len(columns_to_encode))

In [15]:
df = pd.get_dummies(df, drop_first=True, columns=columns_to_encode)

In [ ]:
df['t_stamp'] = pd.to_datetime(df['t_stamp'], format='mixed')
df_22 = df[df['t_stamp'].dt.day == 22].copy()
df_29 = df[df['t_stamp'].dt.day == 29].copy()
start_time_29 = df_29['t_stamp'].min()
end_time_29 = df_29['t_stamp'].max()
start_time_22 = df_22['t_stamp'].min()
end_time_22 = df_22['t_stamp'].max()
print(f"22nd of June started at : {start_time_22} and ended at : {end_time_22}")
print(f"29th of June started at : {start_time_29} and ended at : {end_time_29}")

In [17]:
df_22.set_index('t_stamp', inplace=True)
df_29.set_index('t_stamp', inplace=True)

In [ ]:
scaler = StandardScaler()
df_22_scaled = scaler.fit_transform(df_22)
df_22_scaled = pd.DataFrame(df_22_scaled, columns=df_22.columns)
df_29_scaled = scaler.fit_transform(df_29)
df_29_scaled = pd.DataFrame(df_29_scaled, columns=df_29.columns)
df_scaled = pd.concat([df_22_scaled, df_29_scaled], axis=0)
df_scaled.head()

## Option 1 : Anomaly Detection

### Isolation Forest

In [ ]:
contamination = 0.02
duration = end_time_29 - start_time_29
print(f"We expect to be attacked for a duration of about {(contamination * duration).seconds} seconds")

model = IsolationForest(n_estimators=100, contamination=contamination, random_state=42)

model.fit(df_22_scaled)
df_29.loc[:, "Anomalie"] = model.predict(df_29_scaled)
df_22.loc[:, "Anomalie"] = 1

df_if = pd.concat([df_22, df_29], axis=0)
df_if.head()

In [ ]:
df_if['Anomalie_Label'] = df_if['Anomalie'].apply(lambda x: 'Normal' if x == 1 else 'Anomalie')
df_if['Anomalie_Label'].value_counts()

In [ ]:
sns.countplot(x='Anomalie_Label', data=df_if)
plt.title("Distribution des Anomalies et des Points Normaux")
plt.show()

#### Some examples of easily separable data

In [ ]:
plt.figure(figsize=(12, 6))
sns.boxplot(x='Anomalie_Label', y='AIT201.Pv', data=df_if)
plt.title("Valeurs de AIT201.Pv par Type (Normales vs Anomalies)")
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
sns.boxplot(x='Anomalie_Label', y='FIT201.Pv', data=df_if)
plt.title("Valeurs de FIT201.Pv par Type (Normales vs Anomalies)")
plt.show()

In [709]:
pca = PCA(n_components=2)
reduced_data = pca.fit_transform(df_scaled)

df_if['PCA1'] = reduced_data[:, 0]
df_if['PCA2'] = reduced_data[:, 1]

In [ ]:
plt.figure(figsize=(12, 6))
sns.scatterplot(x='PCA1', y='PCA2', hue='Anomalie_Label', palette='bright', data=df_if)
plt.title("Projection 2D des Anomalies et Points Normaux")
plt.show()

### One Class SVM

In [ ]:
svm = OneClassSVM(nu=0.02, kernel='rbf', gamma=0.001)
svm.fit(df_22_scaled)
df_29.loc[:, "Anomalie"] = svm.predict(df_29_scaled)
df_22.loc[:, "Anomalie"] = 1

df_svm = pd.concat([df_22, df_29], axis=0)
df_svm['Anomalie_Label'] = df_svm['Anomalie'].apply(lambda x: 'Normal' if x == 1 else 'Anomalie')
df_svm['Anomalie_Label'].value_counts()

In [ ]:
df_svm['PCA1'] = reduced_data[:, 0]
df_svm['PCA2'] = reduced_data[:, 1]

plt.figure(figsize=(12, 6))
sns.scatterplot(x='PCA1', y='PCA2', hue='Anomalie_Label', palette='bright', data=df_svm)
plt.title("Projection 2D des Anomalies et Points Normaux (SVM)")
plt.show()

## Option 2 : Adversarial attacks against classification

### Random Forest

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
X = df_if.drop(columns=['Anomalie'])
y = df_if['Anomalie']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

print(clf.score(X_test, y_test))

In [ ]:
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
cm = confusion_matrix(y_test, y_pred)
cp = cm / cm.sum(axis=1)[:, np.newaxis] # Confusion Matrix Percentage
plt.figure(figsize=(10, 7))
sns.heatmap(cp, annot=cm, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('Truth')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
from art.attacks.evasion import BoundaryAttack
from art.estimators.classification import SklearnClassifier

art_model = SklearnClassifier(model=clf)

boundary_attack = BoundaryAttack(estimator=art_model, targeted=False)

X_test_adv = boundary_attack.generate(X_test)

y_pred_adv = clf.predict(X_test_adv)
print("Rapport après attaque adversariale :")
print(classification_report(y_test, y_pred_adv))

### One Class SVM Anomalies

In [ ]:
X = df_svm.drop(columns=['Anomalie_Label'])
y = df_svm['Anomalie_Label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

print(clf.score(X_test, y_test))

In [ ]:
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
cm = confusion_matrix(y_test, y_pred)
cp = cm / cm.sum(axis=1)[:, np.newaxis]
plt.figure(figsize=(10, 7))
sns.heatmap(cp, annot=cm, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('Truth')
plt.title('Confusion Matrix')
plt.show()